# Projeto para Educação - Geração de Exercícios (RAG e Agentes)

* Parte 1: Uso de documentos como referência via RAG com PINECONE
* Parte 2: Tutor digital com agente de IA e acesso ao repositório vetorial


## Instalação das bibliotecas


In [ ]:
import ipywidgets as widgets
from IPython.display import display
from dotenv import find_dotenv, load_dotenv
from langchain_groq import ChatGroq
from pinecone import Pinecone, ServerlessSpec
import os
import time

from langchain_text_splitters import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain_core.documents import Document
from langchain_pinecone import PineconeVectorStore
from langchain_docling import DoclingLoader
from docling.document_converter import DocumentConverter
from langchain.tools.retriever import create_retriever_tool
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from pathlib import Path

from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS


## Escolha do modelo



In [ ]:
load_dotenv(find_dotenv())

In [ ]:


def load_llm(id_model, temperature):
  llm = ChatGroq(
      model=id_model,
      temperature=temperature,
      max_tokens=None,
      timeout=None,
      max_retries=2,
  )
  return llm

In [ ]:
id_model = "deepseek-r1-distill-llama-70b"  # @param {type:"string"}
temperature = 0.7 #@param {type:"slider", min:0.1, max:1.5, step:0.1}

llm = load_llm(id_model, temperature)

## Construindo o prompt

In [ ]:
def format_res(res, return_thinking=False):
  res = res.strip()

  if return_thinking:
    res = res.replace("<think>", "[pensando...] ")
    res = res.replace("</think>", "\n---\n")

  else:
    if "</think>" in res:
      res = res.split("</think>")[-1].strip()

  return res

def show_res(res):
    from IPython.display import Markdown
    display(Markdown(res))

### Chain-of-Thought Prompting



#### Exemplo de exercício 1 - Resolução de Problema de Ciência ou Conhecimentos gerais

In [ ]:
prompt = """
Explique o raciocínio passo a passo antes de responder e detalhe cada fase do processo.
Pergunta: Por que as nuvens se formam no céu?
"""
res = llm.invoke(prompt)
show_res(format_res(res.content, return_thinking=True))

#### Exemplo de exercício 2 - Resolução de Problema de Matemática

In [ ]:
prompt = """
Resolva o seguinte problema de forma passo a passo:
Se João tem 3 vezes mais maçãs que Maria, e juntos eles têm 48 maçãs, quantas maçãs cada um tem?
Pense passo a passo.
"""
res = llm.invoke(prompt)
show_res(format_res(res.content, return_thinking=True))

#### Exemplo de exercício 3 - Análise de Decisão

In [ ]:
prompt = """
Você é um consultor financeiro. Um cliente tem 100 mil reais para investir. Ele pode escolher entre um fundo de ações com alta volatilidade e um título de renda fixa de baixo risco.
Pense passo a passo: quais fatores ele deve considerar para tomar a decisão mais adequada ao seu perfil?
Explique seu raciocínio antes de sugerir uma opção.
"""
res = llm.invoke(prompt)
show_res(format_res(res.content, return_thinking=True))

In [ ]:
prompt = "explique computação quântica para uma criança de 5 anos"
res = llm.invoke(prompt)
show_res(format_res(res.content, return_thinking=True))

# PINECONE - Repositório Vetorial


In [ ]:
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

## Criando uma collection



In [ ]:
def create_index(pc,index_name,dimension=1024):
    existing_indexes= [index_info["name"] for index_info in pc.list_indexes()] # change if desired
    if index_name in existing_indexes:
        print(f"Deletando o índice existente '{index_name}'...")
        pc.delete_index(index_name)
        time.sleep(1) # Aguardar a exclusão

    pc.create_index(
        name=index_name,
        metric="cosine",
        dimension=dimension, # Isso irá retornar 768
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    while not pc.describe_index(index_name).status["ready"]:
        print("Aguardando o índice ficar pronto..."+pc.describe_index(index_name).status)
        time.sleep(1)

    index = pc.Index(index_name)
    return index

### Modelo de embedding



In [ ]:
embedding_model = "BAAI/bge-m3"

embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

In [ ]:

def split_chunks(markdown_splits, chunk_size=1000, chunk_overlap=200):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = text_splitter.split_documents(markdown_splits)
    print(f"Chunks gerados: {len(chunks)}")
    return chunks

## Criação do Retriever


In [ ]:
def config_retriever(pc ,index_name, docs, embeddings):
  create_index(pc, index_name)
  vectorstore = PineconeVectorStore.from_documents(
      documents=docs,
      embedding=embeddings,
      index_name=index_name,
  )
  return vectorstore.as_retriever()

## Carregando documentos



### Funções para divisão (split) do documento em chunks



In [ ]:
def split_chunks(markdown_splits, chunk_size=1000, chunk_overlap=200):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = text_splitter.split_documents(markdown_splits)
    print(f"Chunks gerados: {len(chunks)}")
    return chunks

In [ ]:
text = """
A biblioteca possui atualmente 42998 livros.
"""

docs = [Document(page_content=text)]

#chunks = split_chunks(docs)
chunks = split_chunks(docs, 10, 3)
chunks

### Criação das demais funções



In [ ]:
from langchain_docling import DoclingLoader

loader = DoclingLoader(file_path = "./artigos/astronomia.pdf")
docs = loader.load()
docs

In [ ]:
def parse_document(file_path):
  loader = DoclingLoader(file_path)
  doc = loader.load()
  return doc

def load_documents(input_path):
  input_path = Path(input_path)

  if input_path.is_dir():
    pdf_files = list(input_path.glob("*.pdf"))
  elif input_path.is_file():
    pdf_files = [input_path]
  else:
    raise ValueError("Caminho inválido. Forneça um diretório ou arquivo")

  documents = []
  for file in pdf_files:
    documents.extend(parse_document(file))

  print(f"Documentos carregados: {len(documents)}")
  return documents

def split_markdown(documents):
  splitter = MarkdownHeaderTextSplitter(
      headers_to_split_on=[
          ("#", "Header_1"),
          ("##", "Header_2"),
          ("###", "Header_3")
      ]
  )
  markdown_splits = [
      split
      for doc in documents
      for split in splitter.split_text(doc.page_content)
  ]

  print(f"Splits gerados via Markdown: {len(markdown_splits)}")
  return markdown_splits

def build_chunks(input_path):
  documents = load_documents(input_path)
  markdown_splits = split_markdown(documents)
  chunks = split_chunks(markdown_splits)
  return chunks

In [ ]:
chunks = build_chunks("./artigos/")
chunks

In [ ]:
retriever = config_retriever(pc,"proj-edu", chunks, embeddings)

## Conexão com o repositório vetorial



In [ ]:
vector_store = PineconeVectorStore(
   index=pc.Index("proj-edu"),
   embedding=embeddings,
)

In [ ]:
results = vector_store.similarity_search_with_score(query = "biblioteca", k=3)
for doc, score in results:
  print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

In [ ]:
results = vector_store.similarity_search_with_score(query = "astronomia", k=1)
for doc, score in results:
  print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

### Carregando coleção já criada e retriever



In [ ]:
def get_retriever(index_name, embeddings,chunks):
  vectorstore = PineconeVectorStore.from_documents(
      embedding=embeddings,
      index_name=index_name,
      documents=chunks
      
  )
  return vectorstore.as_retriever(
      search_type = 'mmr',
      search_kwargs = {'k': 6, 'fetch_k': 10}
  )

In [ ]:
retriever = get_retriever("proj-edu", embeddings, chunks)

## Definindo Parâmetros de Personalização Através de Form



In [ ]:
def create_form():
  level = widgets.Dropdown(
      options = ['Iniciante', 'Intermediário', 'Avançado'],
      description = 'Nível',
      value = 'Intermediário'
  )

  topic = widgets.Text(
      description='Tema:',
      placeholder='Matemática, Inglês, Física, Biologia, etc.'
  )

  quantity = widgets.IntSlider(
      value=5,
      min=1,
      max=10,
      step=1,
      description='Qtd Exercícios:',
  )

  interests = widgets.Text(
      description='Interesses ou Preferências:',
      placeholder='Ex: Filmes, Esportes, Jogos, Música, etc....',
  )

  generate_btn = widgets.Button(description="Gerar exercícios")
  export_btn = widgets.Button(description="Exportar (.docx)", disabled=True)
  output = widgets.Output()

  form_fields = {
      'level': level,
      'topic': topic,
      'quantity': quantity,
      'interests': interests,
      'generate_btn': generate_btn,
      'export_btn': export_btn,
      'output': output
  }

  return form_fields

In [ ]:
def display_form(option):
  form = widgets.VBox([
      option['level'],
      option['topic'],
      option['quantity'],
      option['interests'],
      option['generate_btn'],
      option['export_btn'],
      option['output']
  ])
  display(form)

In [ ]:
form = create_form()
display_form(form)

## Geração de exercício COM RAG



### Obtenção do contexto

In [ ]:
def get_context(retriever, topic):
  retrieved_docs = retriever.invoke(topic)
  context = "\n\n".join([doc.page_content for doc in retrieved_docs])
  return context

In [ ]:
context = get_context(retriever, "astrominia")
print(context)

### Construção do template para RAG



In [ ]:
def build_prompt_rag(form):
  quantity = form['quantity'].value
  level = form['level'].value
  interests = form['interests'].value

  prompt = f"""
Gere {quantity} exercícios em português sobre o conteúdo fornecido como contexto abaixo. Nível de dificuldade: {level}.
Cada exercício deve ser de múltipla escolha com 4 alternativas, incluir a resposta correta e uma explicação passo a passo.
Não invente dados externos nem saia do escopo do material apresentado, utilize exclusivamente o conteúdo fornecido.
Para e explicação da resposta, não justifique mencionando que foi obtido com o contexto fornecido abaixo. Você deve justificá-la com base no conhecimento que você tem. gere em portugues
{f"- Apenas caso faça sentido no contexto, adapte de forma natural e sutil os enunciados dos exercícios para refletir a afinidade do aluno com o tema '{interests}'" if interests else ""}
"""
  return prompt

In [ ]:
prompt_rag = build_prompt_rag(form)
prompt_rag

### Função de geração final



In [ ]:
from langchain.prompts import PromptTemplate

def llm_generate(llm, prompt_rag, context):
  prompt_template_rag = """
  {input}
  ---
  Contexto: {context}
  """
  template_rag = PromptTemplate(
      input_variables = ["context", "input"],
      template=prompt_template_rag,
  )

  prompt_llm = prompt_template_rag.format(input=prompt_rag, context=context)
  print(prompt_llm)

  res = llm.invoke(prompt_llm)
  return res

In [ ]:
res = llm_generate(llm, prompt_rag, context)
show_res(format_res(res.content))

## Finalização



In [ ]:
def generate_exercises_rag(b):

  form['output'].clear_output()

  with form['output']:

    prompt_rag = build_prompt_rag(form)
    context = get_context(retriever, form['topic'].value)
    res = llm_generate(llm, prompt_rag, context)
    show_res(format_res(res.content, return_thinking=True))

    form['export_btn'].disabled = False
    global doc_content
    doc_content = res.content

In [ ]:
form = create_form()
form['generate_btn'].on_click(generate_exercises_rag)
#form['export_btn'].on_click(export_doc)

In [ ]:
display_form(form)

---
# Criação de Agentes com LangGraph



In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict
from typing import Annotated
from langchain_core.tools import tool
from langchain.tools.retriever import create_retriever_tool
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
import numexpr
import math

In [ ]:
id_model = "meta-llama/llama-4-maverick-17b-128e-instruct"  # @param {type:"string"}
temperature = 0.7 #@param {type:"slider", min:0.1, max:1.5, step:0.1}

llm = load_llm(id_model, temperature)

## Construindo o State



In [ ]:
class State(TypedDict):
  messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)
graph_builder

## Adicionando nós (nodes)



In [ ]:
def agent(state: State):
  return {"messages": [llm.invoke(state["messages"])]}

graph_builder.add_node("agent", agent)

## Adicionando Entry point



In [ ]:
graph_builder.add_edge(START, "agent")
graph_builder.add_edge("agent", END)

## Compilando e exibindo



In [ ]:
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

## Rodando o chatbot



In [ ]:
while True:
  print("\n===========")
  user_input = input("Usuário: ")
  if user_input.lower() in ["q", "sair"]:
    print("Até mais!")
    break
  for event in graph.stream({"messages": [("user", user_input)]}):
    for value in event.values():
      print("\n=========")
      print("Tutor: ", value["messages"][-1].content)
      

## Integração com Tools (ferramentas)



### Criando uma tool de cálculo matemático



In [ ]:
# "37593 * 67"


@tool
def calculator_tool(expression: str) -> str:
  """Use quando a mensagem pedir explicitamente pela resposta de um cálculo """
  local_dict = {"pi": math.pi, "e": math.e}
  return str(
      numexpr.evaluate(
          expression.strip(),
          global_dict={},
          local_dict=local_dict,
      )
  )

In [ ]:
calculator_tool

In [ ]:
tools = [calculator_tool]
tools_node = ToolNode(tools=tools)

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
def agent(state: State):
  messages = state["messages"]
  response = llm_with_tools.invoke(messages)
  tool_calls = response.additional_kwargs.get("tools_calls")
  if tool_calls is not None:
    print(tool_calls)
  return {"messages": [response]}

In [ ]:
response = llm_with_tools.invoke("quanto é 969611 dividido por 23?")
print(response.tool_calls)

In [ ]:
builder = StateGraph(State)

builder.add_node("agent", agent)
builder.add_node("tools", tools_node)

builder.add_edge(START, "agent")
builder.add_conditional_edges("agent", tools_condition, ["tools", END])
builder.add_edge("tools", "agent")

## Adicionando memória



In [ ]:
memory = MemorySaver()

In [ ]:
graph = builder.compile(checkpointer=memory)

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
def stream_graph_updates(message):
  for event in graph.stream(
      {"messages": message},
      {"configurable": {"thread_id": "1"}},
  ):
    for value in event.values():
      last_message = value["messages"][-1].content
      if len(last_message) > 0:
        print("\n=======\n")
        print("Tutor:", last_message)

In [ ]:
start_msg = "Olá, sou seu tutor digital! Como posso ajudar?"
stream_graph_updates([("assistant", start_msg)])
print(start_msg)



In [ ]:
while True:
    try:
        print("\n==========\n")
        user_input = input("Usuário: ")
        if user_input.lower() in ["q", "sair"]:
            print("Até mais!")
            break

        stream_graph_updates([("user", user_input)])
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        break

## Criando tool para pesquisa



In [ ]:
from langchain_tavily import TavilySearch

@tool
def search_tool(query: str="") -> str:
  """Busca informações na internet com base na consulta fornecida, use quando pedir por informações recentes ou por pesquisa"""
  search_tavily = TavilySearch(max_results = 3)
  search_res = search_tavily.invoke(query)
  return search_res

In [ ]:
search_res = search_tool.invoke("notícias recentes sobre o telescópio james webb")
search_res

## Criando tool para recuperar informações (RAG)



In [ ]:
retriever_tool = create_retriever_tool(
    retriever,
    "retriever_docs",
    "Retorna informações somente quando usuário pedir por informações do banco de dados"
)

In [ ]:
tools = [calculator_tool, retriever_tool, search_tool]
llm_with_tools = llm.bind_tools(tools)

response = llm_with_tools.invoke("retorne informações do banco de dados sobre astronomia")
response.tool_calls

## Organizando código final em funções



In [ ]:
def build_tools(tools):
  return tools, ToolNode(tools)

In [ ]:
def config_graph(agent, tools_node, memory):
    """Cria e configura o grafo do LangGraph com os nós e arestas"""
    builder = StateGraph(State)
    builder.add_node("agent", agent)
    builder.add_node("tools", tools_node)

    builder.add_edge(START, "agent")
    builder.add_conditional_edges("agent", tools_condition, ["tools", END])
    builder.add_edge("tools", "agent")

    return builder.compile(checkpointer=memory)

In [ ]:
def run_agent(graph):
    """Loop principal de interação com o agente."""
    # Mensagem inicial (como se a IA estivesse começando)
    start_msg = "Olá, sou seu tutor digital! Como posso ajudar?"
    stream_graph_updates([("assistant", start_msg)])
    print(start_msg)

    while True:
        try:
            print("\n==========\n")
            user_input = input("Usuário: ")
            if user_input.lower() in ["q", "sair"]:
                print("Até mais!")
                break

            stream_graph_updates([("user", user_input)])
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break

In [ ]:
tools, tools_node = build_tools([calculator_tool, retriever_tool, search_tool])
memory = MemorySaver()
llm_with_tools = llm.bind_tools(tools)

In [ ]:
graph = config_graph(agent, tools_node, memory)
run_agent(graph)